# Textured Cube Example

This notebook demonstrates applying a texture to all faces of a rotating cube using Matplotlib's 3D capabilities. Provide your own image file named `texture.jpg` in the repository root before running.


In [16]:
import subprocess
import sys
from pathlib import Path

import numpy as np
import matplotlib
matplotlib.use("Agg")

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from IPython.display import HTML, display
from PIL import Image

try:
    import imageio.v2 as imageio
except ImportError:  # pragma: no cover
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "imageio", "imageio-ffmpeg"])
    import imageio.v2 as imageio

plt.ioff()

def load_texture(path: Path, target_res: int = 256) -> np.ndarray:
    """Load texture, crop to square, resample, and normalize to [0, 1]."""
    print(f"Loading texture from {path} (target {target_res}px)...")
    image = plt.imread(path)
    if image.ndim == 2:
        image = np.stack([image] * 3, axis=-1)
    if image.dtype != np.float32:
        image = image.astype(np.float32) / 255.0
    height, width = image.shape[:2]
    crop = min(height, width)
    top = (height - crop) // 2
    left = (width - crop) // 2
    image = image[top:top + crop, left:left + crop]
    if crop != target_res:
        pil_image = Image.fromarray(np.clip(image * 255.0, 0, 255).astype(np.uint8))
        image = np.asarray(pil_image.resize((target_res, target_res), Image.LANCZOS), dtype=np.float32) / 255.0
    image = np.clip(image, 0.0, 1.0)
    print(f"Texture ready: {image.shape[0]}x{image.shape[1]} pixels")
    return image



In [17]:
texture_path = Path("texture.jpg")

if not texture_path.exists():
    raise FileNotFoundError("Place texture.jpg in the repository root before running this cell.")

texture_resolution = 256
mesh_resolution = 96
frame_count = 90
frame_rate = 24

print("Preparing cube geometry and figure...")
texture = load_texture(texture_path, texture_resolution)
if texture.shape[0] != mesh_resolution:
    print(f"Resampling texture for mesh to {mesh_resolution}px...")
    texture_surface = np.asarray(Image.fromarray((texture * 255).astype(np.uint8)).resize((mesh_resolution, mesh_resolution), Image.BICUBIC), dtype=np.float32) / 255.0
else:
    texture_surface = texture
grid = np.linspace(-0.5, 0.5, texture_surface.shape[0])
U, V = np.meshgrid(grid, grid)
ones = np.ones_like(U)
print(f"Surface mesh resolution: {U.shape[0]}x{U.shape[1]}")

fig = plt.figure(figsize=(3.8, 3.8), dpi=110)
ax = fig.add_subplot(111, projection="3d")
ax.axis("off")
ax.set_box_aspect((1, 1, 1))
ax.set_xlim(-0.55, 0.55)
ax.set_ylim(-0.55, 0.55)
ax.set_zlim(-0.55, 0.55)
ax.dist = 8
ax.set_facecolor((0.03, 0.03, 0.03))
fig.patch.set_facecolor((0.03, 0.03, 0.03))

faces = [
    ax.plot_surface(U, 0.5 * ones, V, facecolors=texture_surface, rstride=1, cstride=1, shade=False, antialiased=False),
    ax.plot_surface(U, -0.5 * ones, V[:, ::-1], facecolors=texture_surface, rstride=1, cstride=1, shade=False, antialiased=False),
    ax.plot_surface(U, V, 0.5 * ones, facecolors=np.flipud(texture_surface), rstride=1, cstride=1, shade=False, antialiased=False),
    ax.plot_surface(U, V, -0.5 * ones, facecolors=np.flipud(texture_surface), rstride=1, cstride=1, shade=False, antialiased=False),
    ax.plot_surface(0.5 * ones, U, V, facecolors=texture_surface, rstride=1, cstride=1, shade=False, antialiased=False),
    ax.plot_surface(-0.5 * ones, U[:, ::-1], V, facecolors=texture_surface, rstride=1, cstride=1, shade=False, antialiased=False),
]

def set_camera(angle: float) -> None:
    ax.view_init(elev=32, azim=angle)

def render_frame(angle: float) -> np.ndarray:
    set_camera(angle)
    fig.canvas.draw()
    frame = np.asarray(fig.canvas.buffer_rgba(), dtype=np.uint8)
    return frame[..., :3]

print("Figure prepared. Ready to render.")


Texture file texture.jpg not found. Attempting automatic download...
Preparing cube geometry and figure...
Loading texture from texture.jpg (target 256px)...
Texture ready: 256x256 pixels
Resampling texture for mesh to 96px...
Surface mesh resolution: 96x96
Figure prepared. Ready to render.


In [18]:
angles = np.linspace(0.0, 360.0, frame_count, endpoint=False)
expected_seconds = len(angles) / frame_rate
output_dir = Path("exports")
output_dir.mkdir(exist_ok=True)
video_path = output_dir / "textured_cube.mp4"
html_path = output_dir / "textured_cube.html"

print(f"Rendering {len(angles)} frames at {frame_rate} fps (~{expected_seconds:.1f}s clip)...")
with imageio.get_writer(video_path, fps=frame_rate, codec="libx264", bitrate="2M", macro_block_size=1) as writer:
    milestone = max(1, len(angles) // 8)
    for idx, angle in enumerate(angles, start=1):
        frame = render_frame(float(angle))
        writer.append_data(frame)
        if idx == 1 or idx == len(angles) or idx % milestone == 0:
            print(f"  frame {idx}/{len(angles)}", flush=True)

print("Encoding complete. Building HTML preview...")

html_template = f"""<!DOCTYPE html>
<html lang=\"en\">
<head>
<meta charset=\"utf-8\">
<title>Textured Cube Preview</title>
<style>
body {{ background: #111; color: #f5f5f5; font-family: system-ui, sans-serif; display: flex; flex-direction: column; align-items: center; padding: 2rem; }}
video {{ max-width: 640px; width: 100%; border: 1px solid #333; border-radius: 8px; background: #000; }}
a {{ color: #4da3ff; margin-top: 1rem; }}
</style>
</head>
<body>
<video controls loop autoplay muted src=\"{video_path.name}\"></video>
<a href=\"{video_path.name}\" download>Download video</a>
</body>
</html>
"""
html_path.write_text(html_template, encoding="utf-8")

print(f"Saved video to {video_path.resolve()}")
print(f"Saved HTML preview to {html_path.resolve()}")

display(HTML(f'<video controls loop autoplay muted src="{video_path.name}" width="640"></video>'))
display(HTML(f'<a href="{html_path.name}" target="_blank">Open standalone HTML viewer</a>'))

plt.close(fig)


Rendering 90 frames at 24 fps (~3.8s clip)...
  frame 1/90
  frame 11/90
  frame 22/90
  frame 33/90
  frame 44/90
  frame 55/90
  frame 66/90
  frame 77/90
  frame 88/90
  frame 90/90
Encoding MP4 complete. Writing GIF...
GIF export complete.
Encoding complete. Building HTML preview...
Saved GIF to C:\Users\bvarb\vscode_ws\basic-gpu-pipeline-example\exports\textured_cube.gif
Saved video to C:\Users\bvarb\vscode_ws\basic-gpu-pipeline-example\exports\textured_cube.mp4
Saved HTML preview to C:\Users\bvarb\vscode_ws\basic-gpu-pipeline-example\exports\textured_cube.html
